In [1]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
pd.Timestamp('20240101')

In [2]:
start = pd.Timestamp('20240101 00:00:00')
end = pd.Timestamp('20240101 02:00:00')
snapshots = pd.date_range(start=start, end=end, freq='h', inclusive='left')

In [ ]:
fuel_cost = dict(
    coal=8,
    gas=100,
    oil=48,
)
efficiency = dict(
    coal=0.33,
    gas=0.58,
    oil=0.35,
)
power_plants = {
    "SA": {"coal": 35000, "wind": 3000, "gas": 8000, "oil": 2000},
    "MZ": {"hydro": 1200},
}
loads = {
    "SA": 42000,
    "MZ": 650,
}

In [11]:
n = pypsa.Network()
n.set_snapshots(snapshots=snapshots)
buses = ['France']

In [12]:
buses_df = pd.DataFrame({"v_nom" : 380, "carrier" : 'AC', 'x':[25], 'y':[-30.5]},
                    index=buses)

In [13]:
n.import_components_from_dataframe(
    buses_df,
    cls_name="Bus")

In [14]:
load_df = pd.DataFrame(
       data = {'France':[500,800]},
       index=n.snapshots
       )

In [ ]:
load_df

In [15]:
for buse in buses:
    n.add(class_name='Load',
          name=buse+" load",
          bus=buse,
          carrier='electricity',
          p_set=load_df[buse])
# n.madd(
#        class_name="Load",
#        names=n.buses.index + " load",
#        bus=buses,
#        carrier='electricity',
#        p_set=load_df)

In [ ]:
n.loads

In [16]:
p_max_pu_wind = pd.DataFrame(
    data={'France':[0.5,0.5]},
    index=n.snapshots
)

marginal_cost_wind = pd.DataFrame(
    data={'France':[100,50]},
    index=n.snapshots
)
#use a suffix to avoid boilerplate to rename everything
n.madd("Generator",
       buses,
       suffix=' wind',
       bus=buses,
       p_nom_extendable=False,
       marginal_cost=marginal_cost_wind,
       p_nom=[800],
       p_max_pu=p_max_pu_wind)

Index(['France wind'], dtype='object')

In [17]:
p_max_pu_solar = pd.DataFrame(
    data={'France':[0.5,0.5]},
    index=n.snapshots
)

marginal_cost_solar = pd.DataFrame(
    data={'France':[0.1,800]},
    index=n.snapshots
)

n.madd("Generator",
       buses,
       suffix=' solar',
       bus=buses,
       p_nom_extendable=False,
       marginal_cost=marginal_cost_solar,
       p_nom=[900],
       p_max_pu=p_max_pu_solar)

Index(['France solar'], dtype='object')

In [ ]:
# n.add(name='UK2FR',
#       class_name='Link',
#       bus0='UK',
#       bus1='France'
#       )

In [18]:
n.optimize(solver_name="highs")

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at C:\Users\MARY Loïc\AppData\Local\Temp\highs.log.
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 4 primals, 10 duals
Objective: 3.45e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


('ok', 'optimal')

In [19]:
n.generators_t.p

Generator,France wind,France solar
snapshot,,
2024-01-01 00:00:00,50.0,450.0
2024-01-01 01:00:00,400.0,400.0


In [20]:
n.buses_t.marginal_price

Bus,France
snapshot,
2024-01-01 00:00:00,100.0
2024-01-01 01:00:00,800.0


In [ ]:
marginal_cost_wind

In [ ]:
n = pypsa.Network()
n.add("Bus", "SA", y=-30.5, x=25, v_nom=400, carrier="AC")
n.add("Bus", "MZ", y=-18.5, x=35.5, v_nom=400, carrier="AC")
n.add(
    "Generator",
    "MZ hydro",
    bus="MZ",
    carrier="hydro",
    p_nom=1200,  # MW
    marginal_cost=0,  # default
)
for tech, p_nom in power_plants["SA"].items():
    n.add(
        "Generator",
        f"SA {tech}",
        bus="SA",
        carrier=tech,
        efficiency=efficiency.get(tech, 1),
        p_nom=p_nom,
        marginal_cost=fuel_cost.get(tech, 0) / efficiency.get(tech, 1),
    )
n.add(
    "Load",
    "SA electricity demand",
    bus="SA",
    p_set=loads["SA"],
    carrier="electricity",
)
n.add(
    "Load",
    "MZ electricity demand",
    bus="MZ",
    p_set=loads["MZ"],
    carrier="electricity",
)
n.add(
    "Line",
    "SA-MZ",
    bus0="SA",
    bus1="MZ",
    s_nom=500,
    x=1,
    r=1,
)
n.optimize(solver_name="highs")

In [ ]:
n.generators_t

In [ ]:
network = pypsa.Network()
network.set_snapshots(range(10))
network.import_series_from_dataframe(
        pd.DataFrame(np.random.rand(10, 4),
            columns=network.generators.index,
                        index=range(10)),
                    "Generator",
                    "p_max_pu")